In [1]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv("Resources/crypto_market_data.csv")

# Display sample data
df_market_data.head(10)

,coin_id,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
0,bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
1,ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
2,tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
3,ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
4,bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
5,binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
6,chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
7,cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756
8,litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408
9,bitcoin-cash-sv,0.92530,3.29641,-1.86656,2.88926,-24.87434,7.42562,93.73082


In [3]:
# Generate summary statistics
df_market_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [4]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

---

### Prepare the Data

In [30]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
scaler = StandardScaler()
columns_to_scale = [
    'price_change_percentage_24h',
    'price_change_percentage_7d',
    'price_change_percentage_14d'
]
scaled_data = scaler.fit_transform(df_market_data[columns_to_scale])

In [31]:
df_market_data.columns

Index(['coin_id', 'price_change_percentage_24h', 'price_change_percentage_7d',
       'price_change_percentage_14d', 'price_change_percentage_30d',
       'price_change_percentage_60d', 'price_change_percentage_200d',
       'price_change_percentage_1y'],
      dtype='object')

In [32]:
# Create a DataFrame with the scaled data
crypto_scaled_data = df_market_data[columns_to_scale].copy()
scaled_df = pd.DataFrame(scaled_data, columns=columns_to_scale)

# Copy the crypto names from the original DataFrame
scaled_df['coin_id'] = df_market_data['coin_id']

# Set the coin_id column as index
scaled_df = scaled_df.set_index('coin_id')

# Display the scaled DataFrame
scaled_df.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d
coin_id,,,
bitcoin,0.508529,0.493193,0.772200
ethereum,0.185446,0.934445,0.558692
tether,0.021774,-0.706337,-0.021680
ripple,-0.040764,-0.810928,0.249458
bitcoin-cash,1.193036,2.000959,1.760610


---

### Find the Best Value for k Using the Original Scaled DataFrame.

In [33]:
# Create a list with the number of k-values from 1 to 11
k_values = list(range(1, 11))

In [34]:
# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list
from sklearn.cluster import KMeans

for k in k_values:
    model = KMeans(n_clusters=k, random_state=1)
    model.fit(scaled_df)
    inertia.append(model.inertia_)

c:\Users\malco\anaconda3\envs\Analytics\lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
c:\Users\malco\anaconda3\envs\Analytics\lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
c:\Users\malco\anaconda3\envs\Analytics\lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
c:\Users\malco\anaconda3\envs\Analytics\lib\site-packages\sklearn\cluster\_kmeans.py:1429: 

In [35]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {
    "k": k_values,
    "inertia": inertia
}

# Create a DataFrame with the data to plot the Elbow curve
elbow_df = pd.DataFrame(elbow_data)

In [36]:
# Plot a line chart with all the inertia values computed with
# the different values of k to visually identify the optimal value for k.
elbow_df.hvplot.line(x='k', y='inertia', title='Elbow Curve', xticks=k_values)

:Curve   [k]   (inertia)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** 4

---

### Cluster Cryptocurrencies with K-means Using the Original Scaled DataFrame

In [37]:
# Initialize the K-Means model using the best value for k
best_k = 4
model = KMeans(n_clusters=best_k, random_state=1)

In [38]:
# Fit the K-Means model using the scaled DataFrame
model.fit(scaled_df)

c:\Users\malco\anaconda3\envs\Analytics\lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


KMeans(n_clusters=4, random_state=1)

In [39]:
# Predict the clusters to group the cryptocurrencies using the scaled DataFrame
kmeans_predictions = model.predict(scaled_df)

# Print the resulting array of cluster values.
print(kmeans_predictions)


[1 1 0 0 2 1 2 1 1 0 0 0 0 2 0 1 0 0 2 0 0 1 0 0 0 0 0 0 1 0 0 0 3 1 0 0 1
 0 1 0 0]


In [40]:
# Create a copy of the scaled DataFrame
clustered_scaled_df = scaled_df.copy()

In [41]:
# Add a new column to the copy of the scaled DataFrame with the predicted clusters
clustered_scaled_df['kmeans_clusters'] = kmeans_predictions

# Display the copy of the scaled DataFrame
clustered_scaled_df.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,kmeans_clusters
coin_id,,,,
bitcoin,0.508529,0.493193,0.772200,1
ethereum,0.185446,0.934445,0.558692,1
tether,0.021774,-0.706337,-0.021680,0
ripple,-0.040764,-0.810928,0.249458,0
bitcoin-cash,1.193036,2.000959,1.760610,2


In [42]:
# Create a scatter plot using hvPlot by setting
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`.
# Color the graph points with the labels found using K-Means and
# add the crypto name in the `hover_cols` parameter to identify
# the cryptocurrency represented by each data point.
clustered_scaled_df.hvplot.scatter(
    x='price_change_percentage_24h',
    y='price_change_percentage_7d',
    by='kmeans_clusters',
    hover_cols=['coin_id'],
    title='Cryptocurrency Clusters Using K-Means'
)

:NdOverlay   [kmeans_clusters]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)

---

### Optimize Clusters with Principal Component Analysis.

In [46]:
# Create a PCA model instance and set `n_components=3`.
from sklearn.decomposition import PCA

pca = PCA(n_components=3)


In [47]:
# Use the PCA model with `fit_transform` to reduce the original scaled DataFrame
# down to three principal components.
pca_data = pca.fit_transform(scaled_df)

# View the scaled PCA data
pca_data[:5]

array([[ 1.03076861,  0.11073879,  0.15293142],
       [ 1.02777426, -0.32436134, -0.24121441],
       [-0.44421197,  0.31903193,  0.44805162],
       [-0.35680243,  0.25945523,  0.72585836],
       [ 2.90921721, -0.08466974, -0.23707831]])

In [48]:
# Retrieve the explained variance to determine how much information
# can be attributed to each principal component.
pca.explained_variance_ratio_

array([0.56214379, 0.2883511 , 0.14950511])

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** The total explained variance of the three principal components is approximately 1.0, or 100%. This means that the PCA transformation captures the entirety of the original data’s variability across the three dimensions:

PC1: ~56.2% of the variance

PC2: ~28.8%

PC3: ~14.9%

Together, they account for 100% of the information in the original 3-feature dataset.

In [49]:
# Create a new DataFrame with the PCA data.
pca_df = pd.DataFrame(
    pca_data,
    columns=["PC1", "PC2", "PC3"]
)

# Copy the crypto names from the original scaled DataFrame
pca_df['coin_id'] = scaled_df.index

# Set the coin_id column as index
pca_df = pca_df.set_index('coin_id')

# Display the scaled PCA DataFrame
pca_df.head()

,PC1,PC2,PC3
coin_id,,,
bitcoin,1.030769,0.110739,0.152931
ethereum,1.027774,-0.324361,-0.241214
tether,-0.444212,0.319032,0.448052
ripple,-0.356802,0.259455,0.725858
bitcoin-cash,2.909217,-0.084670,-0.237078


---

### Find the Best Value for k Using the Scaled PCA DataFrame

In [51]:
# Create a list with the number of k-values from 1 to 11
pca_k_values = list(range(1, 11))

In [52]:
# Create an empty list to store the inertia values
pca_inertia = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_pca`
# 3. Append the model.inertia_ to the inertia list
for k in pca_k_values:
    pca_model = KMeans(n_clusters=k, random_state=1)
    pca_model.fit(pca_df)
    pca_inertia.append(pca_model.inertia_)

c:\Users\malco\anaconda3\envs\Analytics\lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
c:\Users\malco\anaconda3\envs\Analytics\lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
c:\Users\malco\anaconda3\envs\Analytics\lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
c:\Users\malco\anaconda3\envs\Analytics\lib\site-packages\sklearn\cluster\_kmeans.py:1429: 

In [53]:
# Create a dictionary with the data to plot the Elbow curve
pca_elbow_data = {
    "k": pca_k_values,
    "inertia": pca_inertia
}

# Create a DataFrame with the data to plot the Elbow curve
pca_elbow_df = pd.DataFrame(pca_elbow_data)


In [54]:
# Plot a line chart with all the inertia values computed with
# the different values of k to visually identify the optimal value for k.
pca_elbow_df.hvplot.line(
    x='k',
    y='inertia',
    title='PCA Elbow Curve',
    xticks=pca_k_values
)


:Curve   [k]   (inertia)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** Based on the elbow curve generated from the PCA-transformed data, the optimal value for k appears to be 4, where the inertia begins to level off noticeably. This suggests that 4 clusters effectively balance compactness and simplicity.


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** No, the optimal k value using PCA (4) is the same as the value identified using the original scaled data. This consistency supports the robustness of the clustering structure — the PCA dimensionality reduction retained the essential patterns of the original feature space.

### Cluster Cryptocurrencies with K-means Using the Scaled PCA DataFrame

In [55]:
# Initialize the K-Means model using the best value for k
pca_kmeans_model = KMeans(n_clusters=4, random_state=1)

In [56]:
# Fit the K-Means model using the PCA data
pca_kmeans_model.fit(pca_df)


c:\Users\malco\anaconda3\envs\Analytics\lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


KMeans(n_clusters=4, random_state=1)

In [57]:
# Predict the clusters to group the cryptocurrencies using the scaled PCA DataFrame
pca_kmeans_predictions = pca_kmeans_model.predict(pca_df)

# Print the resulting array of cluster values.
print(pca_kmeans_predictions)


[1 1 0 0 2 1 2 1 1 0 0 0 0 2 0 1 0 0 2 0 0 1 0 0 0 0 0 0 1 0 0 0 3 1 0 0 1
 0 1 0 0]


In [59]:
# Create a copy of the scaled PCA DataFrame
clustered_pca_df = pca_df.copy()

# Add a new column to the copy of the PCA DataFrame with the predicted clusters
clustered_pca_df['kmeans_clusters'] = pca_kmeans_predictions

# Display the copy of the scaled PCA DataFrame
clustered_pca_df.head()


,PC1,PC2,PC3,kmeans_clusters
coin_id,,,,
bitcoin,1.030769,0.110739,0.152931,1
ethereum,1.027774,-0.324361,-0.241214,1
tether,-0.444212,0.319032,0.448052,0
ripple,-0.356802,0.259455,0.725858,0
bitcoin-cash,2.909217,-0.084670,-0.237078,2


In [60]:
# Create a scatter plot using hvPlot by setting
# `x="PC1"` and `y="PC2"`.
# Color the graph points with the labels found using K-Means and
# add the crypto name in the `hover_cols` parameter to identify
# the cryptocurrency represented by each data point.
clustered_pca_df.hvplot.scatter(
    x='PC1',
    y='PC2',
    by='kmeans_clusters',
    hover_cols=['coin_id'],
    title='Cryptocurrency Clusters Using PCA + K-Means'
)

:NdOverlay   [kmeans_clusters]
   :Scatter   [PC1]   (PC2,coin_id)

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [61]:
# Composite plot to contrast the Elbow curves
elbow_df_plot = elbow_df.hvplot.line(x='k', y='inertia', title='Elbow Curve Comparison')
pca_elbow_df_plot = pca_elbow_df.hvplot.line(x='k', y='inertia', title='Elbow Curve Comparison (PCA)')

elbow_df_plot + pca_elbow_df_plot

:Layout
   .Curve.I  :Curve   [k]   (inertia)
   .Curve.II :Curve   [k]   (inertia)

In [62]:
# Composite plot to contrast the clusters
original_clusters_plot = clustered_scaled_df.hvplot.scatter(
    x='price_change_percentage_24h',
    y='price_change_percentage_7d',
    by='kmeans_clusters',
    hover_cols=['coin_id'],
    title='Clusters from Original Scaled Data'
)

pca_clusters_plot = clustered_pca_df.hvplot.scatter(
    x='PC1',
    y='PC2',
    by='kmeans_clusters',
    hover_cols=['coin_id'],
    title='Clusters from PCA-Reduced Data'
)

original_clusters_plot + pca_clusters_plot

:Layout
   .NdOverlay.I  :NdOverlay   [kmeans_clusters]
      :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)
   .NdOverlay.II :NdOverlay   [kmeans_clusters]
      :Scatter   [PC1]   (PC2,coin_id)

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** Using fewer features through PCA simplifies the dataset and allows for easier visualization of clusters. While the PCA-based clustering preserved the general groupings found in the original data, some detail may be lost due to dimensionality reduction.

The impact of using fewer features is a trade-off:

Pros: It reduces noise and redundancy, making patterns clearer and more interpretable.

Cons: Some subtle distinctions between data points might be blurred, potentially affecting clustering precision.

In this case, the PCA-based clusters aligned closely with those from the full feature set, suggesting that PCA retained the essential structure of the data while enabling a more compact and efficient clustering process.